# Training PerAct Agent on handoversim data

In [ ]:
import numpy as np
np.bool = np.bool_ # bad trick to fix numpy version issue :(
import os
import sys
from natsort import natsorted

sys.path = [p for p in sys.path if '/peract/' not in p]

# Set `PYOPENGL_PLATFORM=egl` for pyrender visualizations
os.environ["DISPLAY"] = ":0"
os.environ["PYOPENGL_PLATFORM"] = "egl"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,3" # Depends on your computer and available GPUs

In [ ]:
import itertools

available_cameras = [f"view_{camera_i}" for camera_i in range(3)]
# Grid search
grid = {
    'fill_replay_setting': ["standard"],#, "standard", "uniform"],
    'cameras': [available_cameras],# [available_cameras[0]]],
    'RGB_AUGMENTATION': ['None','partial','full'],
    'demo_augm_n': [5],
    'keypoint_approach': [True],#, False],
    'only_learn_approach': [True]
}
# Loop over al grid search combinations
counter = 0
lst_settings = []
for values in itertools.product(*grid.values()):
    
    point = dict(zip(grid.keys(), values))
    # merge the general settings
    settings = {**point}
    lst_settings.append(settings)
    print(counter, settings)
    counter += 1

In [ ]:
from notebook_helpers.build_replay import load_replay_buffer
from notebook_helpers.build_training import build_agent, agent_training

# Loop over al grid search combinations: and run
for run_settings in lst_settings:
    print(run_settings)

    train_data_iter, test_data_iter = load_replay_buffer(run_settings)
    peract_agent = build_agent(run_settings)
    agent_training(run_settings, peract_agent, train_data_iter, test_data_iter)

In [ ]:
## For single (test) run
# available_cameras = [f"view_{camera_i}" for camera_i in range(3)]
# run_settings = {
#     'fill_replay_setting': "crop",
#     'cameras': available_cameras,
#     'RGB_AUGMENTATION': 'partial',
#     'keypoint_approach': True,
#     'demo_augm_n': 5,
#     'only_learn_approach': True
# }
# print(run_settings)

# train_data_iter, test_data_iter = load_replay_buffer(run_settings)
# peract_agent = build_agent(run_settings)
# agent_training(run_settings, peract_agent, train_data_iter, test_data_iter)